In [1]:
from apperception.database import database

In [2]:
cameras = [
    'CAM_FRONT',
    'CAM_FRONT_LEFT',
    'CAM_FRONT_RIGHT',
    'CAM_BACK_LEFT',
    'CAM_BACK_RIGHT',
    'CAM_BACK',
]

In [3]:
scenes = database.sql("""
select distinct cameraId
from cameras
""")
scenes = scenes['cameraid'].tolist()
scenes

['scene-0061',
 'scene-0103',
 'scene-0553',
 'scene-0655',
 'scene-0757',
 'scene-0796',
 'scene-0916',
 'scene-1077',
 'scene-1094',
 'scene-1100']

In [4]:
videos = {}
for scene in scenes:
    for camera in cameras:
        output = database._execute_query(f"""
        select
            cameraId,
            frameId,
            frameNum,
            filename,
            ARRAY [st_x(cameraTranslation), st_y(cameraTranslation), st_z(cameraTranslation)] as cameraTranslation,
            cameraRotation,
            cameraIntrinsic,
            ARRAY [st_x(egoTranslation), st_y(egoTranslation), st_z(egoTranslation)] as egoTranslation,
            egoRotation,
            timestamp,
            cameraHeading,
            egoHeading,
            roadDirection::real
        from cameras
        where
            cameraId = '{scene}' AND
            filename like '%/{camera}/%'
        order by timestamp
        """)
        videos[f'{scene}-{camera}'] = output  # [(f, t.to_pydatetime()) for f, t in output.itertuples(index=False, name=None)]

In [5]:
videos['scene-0061-CAM_FRONT'][:10]

[('scene-0061',
  'e3d495d4ac534d54b321f50006683844',
  2,
  'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
  [410.87244105365085, 1179.57081167158, 1.4936775157046178],
  [-0.12889418, -0.6929588, -0.11984578, -0.6991648],
  [[1266.4172, 0.0, 816.267], [0.0, 1266.4172, 491.50708], [0.0, 0.0, 1.0]],
  [411.4199861830012, 1181.197175631848, 0.0],
  [0.57200634, -0.0021434845, 0.011564095, -0.82016486],
  datetime.datetime(2018, 7, 23, 20, 28, 47, 612460, tzinfo=datetime.timezone.utc),
  160.10384,
  159.7781,
  159.778),
 ('scene-0061',
  '68e8e98cf7b0487baa139df808641db7',
  7,
  'sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927662460.jpg',
  [410.70977105698245, 1179.137526706944, 1.4912453285726386],
  [-0.12866224, -0.6924895, -0.11990313, -0.69966245],
  [[1266.4172, 0.0, 816.267], [0.0, 1266.4172, 491.50708], [0.0, 0.0, 1.0]],
  [411.2579065092736, 1180.7659224870931, 0.0],
  [0.57210505, -0.0015150002, 0.011903272, -0.8200

In [6]:
fps = 20

In [7]:
t_start = videos['scene-0061-CAM_FRONT'][0][9]
t_end = videos['scene-0061-CAM_FRONT'][66][9]
l = len(videos['scene-0061-CAM_FRONT']) - 2
(t_end - t_start).total_seconds(), (66) / 12

(5.55, 5.5)

In [8]:
prev = videos['scene-0061-CAM_FRONT'][0][9]
i = 1
a = set()
for frame in videos['scene-0061-CAM_FRONT'][1:]:
    a.add((frame[9] - prev).total_seconds())
    print(i, (frame[9] - prev).total_seconds(), frame[3][:10])
    prev = frame[9]
    i += 1
print(sorted([*a]))

1 0.05 sweeps/CAM
2 0.1 sweeps/CAM
3 0.1 sweeps/CAM
4 0.050017 sweeps/CAM
5 0.099983 sweeps/CAM
6 0.1 samples/CA
7 0.05 sweeps/CAM
8 0.1 sweeps/CAM
9 0.1 sweeps/CAM
10 0.05 sweeps/CAM
11 0.1 sweeps/CAM
12 0.1 sweeps/CAM
13 0.05 samples/CA
14 0.1 sweeps/CAM
15 0.1 sweeps/CAM
16 0.05 sweeps/CAM
17 0.1 sweeps/CAM
18 0.1 sweeps/CAM
19 0.05 samples/CA
20 0.1 sweeps/CAM
21 0.1 sweeps/CAM
22 0.05 sweeps/CAM
23 0.1 sweeps/CAM
24 0.1 sweeps/CAM
25 0.05 samples/CA
26 0.1 sweeps/CAM
27 0.101322 sweeps/CAM
28 0.048678 sweeps/CAM
29 0.1 sweeps/CAM
30 0.1 samples/CA
31 0.05 sweeps/CAM
32 0.1 sweeps/CAM
33 0.1 sweeps/CAM
34 0.05 sweeps/CAM
35 0.1 sweeps/CAM
36 0.1 samples/CA
37 0.05 sweeps/CAM
38 0.1 sweeps/CAM
39 0.1 sweeps/CAM
40 0.05 sweeps/CAM
41 0.100026 sweeps/CAM
42 0.099974 sweeps/CAM
43 0.05 samples/CA
44 0.1 sweeps/CAM
45 0.1 sweeps/CAM
46 0.05 sweeps/CAM
47 0.1 sweeps/CAM
48 0.1 sweeps/CAM
49 0.05 samples/CA
50 0.1 sweeps/CAM
51 0.1 sweeps/CAM
52 0.05 sweeps/CAM
53 0.1 sweeps/CAM
54 0.1 sw

In [9]:
import os
if 'NUSCENE_DATA' in os.environ:
    BASE_DIR = os.environ['NUSCENE_DATA']
else:
    BASE_DIR = '/work/apperception/data/nuScenes/full-dataset-v1.0/Mini'
BASE_DIR

'/Users/chanwutk/Documents/apperception/data/v1.0-mini'

In [10]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2

Frame = tuple

def frame_with_best_alignments(scene: "List[Frame]") -> "Frame":
    max_aligned_frames = -1
    frame_idx = None
    frame_t = None
    for idx, frame in enumerate(scene):
        t = frame[9]
        aligned_frames = 0
        for _frame in scene:
            _t = _frame[9]
            sec_diff = (t - _t).total_seconds()
            if (sec_diff * fps).is_integer():
                aligned_frames += 1
        if aligned_frames > max_aligned_frames:
            max_aligned_frames = aligned_frames
            frame_idx = idx
            frame_t = t
    
    return frame_idx, frame_t


def compose(name: str, scene: "List[Frame]"):
    t_start = scene[0][9]
    t_end = scene[-1][9]

    frame_idx, frame_t = frame_with_best_alignments(scene)
    
    frames_before = int((frame_t - t_start).total_seconds() * fps)
    
    def get_timestamp(idx: int):
        return frame_t - timedelta(seconds=(frames_before - idx) / fps)
    
    assert get_timestamp(0) >= t_start, f'{get_timestamp(0)} {t_start}'
    
    frames: "List[npt.NDArray]" = []
    i = 0
    frame_idx = 0
    while get_timestamp(i) < t_end:
        t = get_timestamp(i)
        while scene[frame_idx + 1][9] < t:
            frame_idx += 1
        f_curr, t_curr = scene[frame_idx][3::6]
        f_next, t_next = scene[frame_idx + 1][3::6]
        
        img_curr = cv2.imread(os.path.join(BASE_DIR, f_curr))
        img_next = cv2.imread(os.path.join(BASE_DIR, f_next))
        
        ratio = (t - t_curr).total_seconds() / (t_next - t_curr).total_seconds()
#         print(ratio)
        
        if ratio < 0.5:
            img = img_curr
        else:
            img = img_next
        frames.append(img)
        
        i += 1
    
    t0 = get_timestamp(0)
    t0_tuple = (t0.year, t0.month, t0.day, t0.hour, t0.minute, t0.second, t0.microsecond)
    filename = f'{name}-{"-".join([*map(str, t0_tuple)])}.mp4'
    out = cv2.VideoWriter(
        os.path.join(BASE_DIR, 'videos', filename),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (1600, 900)
    )
    for frame in frames:
        out.write(frame)
    
    return frames, filename, get_timestamp(0)

In [11]:
import pickle
output = {}
for name in videos:
    video = videos[name]
    frames, filename, start = compose(name, video)
    output[name] = {
        "filename": filename,
        "start": start,
        "columns": [
            "cameraId",
            "frameId",
            "frameNum",
            "filename",
            "cameraTranslation",
            "cameraRotation",
            "cameraIntrinsic",
            "egoTranslation",
            "egoRotation",
            "timestamp",
            "cameraHeading",
            "egoHeading",
            "roadDirection",
        ],
        "frames": video
    }
    print(filename)
with open(os.path.join(BASE_DIR, 'videos', 'frames.pickle'), "wb") as f:
    pickle.dump(output, f)

scene-0061-CAM_FRONT-2018-7-23-20-28-47-612460.mp4
scene-0061-CAM_FRONT_LEFT-2018-7-23-20-28-47-604844.mp4
scene-0061-CAM_FRONT_RIGHT-2018-7-23-20-28-47-620339.mp4
scene-0061-CAM_BACK_LEFT-2018-7-23-20-28-47-647423.mp4
scene-0061-CAM_BACK_RIGHT-2018-7-23-20-28-47-627893.mp4
scene-0061-CAM_BACK-2018-7-23-20-28-47-637525.mp4
scene-0103-CAM_FRONT-2018-8-1-12-26-43-512404.mp4
scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4
scene-0103-CAM_FRONT_RIGHT-2018-8-1-12-26-43-520482.mp4
scene-0103-CAM_BACK_LEFT-2018-8-1-12-26-43-547405.mp4
scene-0103-CAM_BACK_RIGHT-2018-8-1-12-26-43-528113.mp4
scene-0103-CAM_BACK-2018-8-1-12-26-43-537558.mp4
scene-0553-CAM_FRONT-2018-8-28-13-48-16-12404.mp4
scene-0553-CAM_FRONT_LEFT-2018-8-28-13-48-16-4799.mp4
scene-0553-CAM_FRONT_RIGHT-2018-8-28-13-48-16-20482.mp4
scene-0553-CAM_BACK_LEFT-2018-8-28-13-48-16-47405.mp4
scene-0553-CAM_BACK_RIGHT-2018-8-28-13-48-16-28113.mp4
scene-0553-CAM_BACK-2018-8-28-13-48-16-37558.mp4
scene-0655-CAM_FRONT-2018-8-27-8-51-32